<a href="https://colab.research.google.com/github/zhgjenny93/NLP-Thinkful/blob/main/Text_Generation_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In the following assignments, you'll use Twitter US Airline Sentiment data. Access the dataset from the Thinkful database using the following credentials:

```
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'twitter_sentiment'
```

The data is in the table called twitter.

1. First, do some data preprocessing to clean up the data as necessary. You can use your solution to the assignment of the Text preprocessing checkpoint.
2. Train a Markov chain model by using only the negative sentiment tweets. Generate some random sentences. Do the generated sentences exhibit the same negative sentiment?
3. Repeat the previous task, but this time, use only the positive sentiment tweets. Generate some random sentences and observe whether they exhibit positive sentiment or not.
4. Now, train your Markov chain model using all of the tweets. Generate some random sentences. Can you say something systematic about the sentiments of the generated tweets?

In [ ]:
pip install markovify

In [ ]:
from sqlalchemy import create_engine
import numpy as np
import pandas as pd
import nltk
from time import time
import re
import spacy
import markovify
import warnings
warnings.filterwarnings('ignore')

nltk.download('gutenberg')

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

In [ ]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'twitter_sentiment'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

tweets_df = pd.read_sql_query('select * from twitter',con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()


tweets_df.head(10)

,index,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,0,570306133677760513,neutral,1.0000,None,NaN,Virgin America,None,cairdin,None,0,@VirginAmerica What @dhepburn said.,None,2015-02-24 11:35:52 -0800,None,Eastern Time (US & Canada)
1,1,570301130888122368,positive,0.3486,None,0.0000,Virgin America,None,jnardino,None,0,@VirginAmerica plus you've added commercials t...,None,2015-02-24 11:15:59 -0800,None,Pacific Time (US & Canada)
2,2,570301083672813571,neutral,0.6837,None,NaN,Virgin America,None,yvonnalynn,None,0,@VirginAmerica I didn't today... Must mean I n...,None,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,None,jnardino,None,0,@VirginAmerica it's really aggressive to blast...,None,2015-02-24 11:15:36 -0800,None,Pacific Time (US & Canada)
4,4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,None,jnardino,None,0,@VirginAmerica and it's a really big bad thing...,None,2015-02-24 11:14:45 -0800,None,Pacific Time (US & Canada)
5,5,570300767074181121,negative,1.0000,Can't Tell,0.6842,Virgin America,None,jnardino,None,0,@VirginAmerica seriously would pay $30 a fligh...,None,2015-02-24 11:14:33 -0800,None,Pacific Time (US & Canada)
6,6,570300616901320704,positive,0.6745,None,0.0000,Virgin America,None,cjmcginnis,None,0,"@VirginAmerica yes, nearly every time I fly VX...",None,2015-02-24 11:13:57 -0800,San Francisco CA,Pacific Time (US & Canada)
7,7,570300248553349120,neutral,0.6340,None,NaN,Virgin America,None,pilot,None,0,@VirginAmerica Really missed a prime opportuni...,None,2015-02-24 11:12:29 -0800,Los Angeles,Pacific Time (US & Canada)
8,8,570299953286942721,positive,0.6559,None,NaN,Virgin America,None,dhepburn,None,0,"@virginamerica Well, I didn't…but NOW I DO! :-D",None,2015-02-24 11:11:19 -0800,San Diego,Pacific Time (US & Canada)
9,9,570295459631263746,positive,1.0000,None,NaN,Virgin America,None,YupitsTate,None,0,"@VirginAmerica it was amazing, and arrived an ...",None,2015-02-24 10:53:27 -0800,Los Angeles,Eastern Time (US & Canada)


In [ ]:
nlp = spacy.load('en')

nlp.max_length = 20000000

twitter_doc = nlp(" ".join(tweets_df.text))

In [ ]:
tweets_df.airline_sentiment.value_counts()

negative    9178
neutral     3099
positive    2363
Name: airline_sentiment, dtype: int64

## 2. Train Markov chain model with only negative sentiment tweets

In [ ]:
neg_tweets_doc = nlp(' '.join(tweets_df[tweets_df['airline_sentiment']=='negative'].text))

neg_tweet_sents = ' '.join([sent.text for sent in neg_tweets_doc.sents if len(sent.text) > 1])

neg_tweet_generator = markovify.Text(neg_tweet_sents, state_size=3)

In [ ]:
# Print three randomly generated sentences
for i in range(3):
  print(neg_tweet_generator.make_sentence())

# Print three randomly generated sentences of no more than 100 characters
for i in range(3):
  print(neg_tweet_generator.make_short_sentence(100))

My mother is onboard &amp; the crew was escorted from the back to 1st class.
Now there's a problem &amp; I don't want to seem crazy!
@JetBlue @fllairport best way to do this online.
Seems like you guys have any working planes?
@JetBlue #fail My wife on the phone just goes to a busy signal.
@united Booked a flight home due to extreme disorganization and lack of snacks on the plane!


The generated sentences definitely exhibit negative sentiment.

## 3. Train Markov chain model now with only positive sentiments

In [ ]:
positive_tweets_doc = nlp(' '.join(tweets_df[tweets_df['airline_sentiment']=='positive'].text))

positive_tweet_sents = ' '.join([sent.text for sent in positive_tweets_doc.sents if len(sent.text) > 1])

positive_tweet_generator = markovify.Text(positive_tweet_sents, state_size=3)

In [ ]:
# Print three randomly generated sentences
for i in range(3):
  print(positive_tweet_generator.make_sentence())

# Print three randomly generated sentences of no more than 100 characters
for i in range(3):
  print(positive_tweet_generator.make_short_sentence(100))

Also looks like you opened some up on my flight right now, thanks!! @USAirways We're back at a gate.
None
Just wanted to see if you have any idea what tail is operating flt 606 2maro?
☕✈ 👍 @JetBlue awesome, thanks! on hold now.
And thanks for being so good to me.
I ❤ your company and your staff and crew were purely amazing!


The generated tweets generally exhibit positive sentiments.

## Train Markov chain model using all tweets. Can you say something systematic about the sentiments of the generated tweets?

In [ ]:
tweet_sents = ' '.join([sent.text for sent in twitter_doc.sents if len(sent.text) > 1])

tweet_generator = markovify.Text(tweet_sents, state_size=3)

In [ ]:
# Print ten randomly generated sentences
for i in range(10):
  print(tweet_generator.make_sentence())

# Print ten randomly generated sentences with no more than 100 characters
for i in range(10):
  print(tweet_generator.make_short_sentence(100))

You guys 2 hrs on hold along w 2 Cancelled Flightled flights this morning and had been on hold for over an hour @USAirways thanks so much!
Thank y'all for being an amazing airline who knows how to treat people with respect and just a little dignity.
#RefundProcedureNotTooPainful I know that but that doesn't get me my money back @AmericanAir I Cancelled Flighted my flight.
Basic customer service @AmericanAir I have tried to change our flight THREE times on the same plane?
@AmericanAir thanks for the bullshit.
@USAirways who would like to know what's going on with US Airways flight as myself for the same price.
Can I Cancelled Flight my car reservation and it came out to more money... why did you Cancelled Flight my flight within the 24hr window but have been trying to reach an agent via phone.
In order to prevent DVT I have to collect my bag and through the TSA checkpoint, I guarantee I will be there alone a day and a half and your phones hang up on me.
They called #United for an altern

There is a mix of all neutral, positive, and negative sentiments. However, it appears that they are mostly negative tweets. This is expected and probably explained as most of the tweets do carry negative sentiment.